### ***Descarga de información de la API de ESIOS***
----
Resumen:
1) Conexion a API de ESIOS


***0) Imports, funciones y variables genericas***

In [ ]:
import requests
import pandas as pd
import warnings
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

In [ ]:
def data_indicator_prices(url:str, id_esios:int, mkt:str, ses:int, headers:dict, date_start:date, date_end:date):

    """
    Funcion para obtener en formato DataFrame los precios de las sesiones spot e intradiarias de España de la API de ESIOS

    Args:
    -   url (str): Cadena de texto de la URL donde realizaremos el REQUEST
    -   id_esios (int): Numero del indicador de la info necesaria de analisis
    -   mkt (str): Señal de mercado
    -   ses (int): Señal de sesion
    -   headers (dict): Diccionario con la info y token necesarios para poder realizar el REQUEST
    -   date_start (datetime.date): Fecha de inicio del intervalo para recopilar informacion
    -   date_end (datetime.date): Fecha de fin del intervalo para recopilar informacion

    Returns:
    -    df_data (pd.dataframe): Dataframe con la informacion de fecha-hora, mercado, sesion y precios
    """

    # Check de las sesiones superiores a la 4 para no cargar informacion debido a la reduccion de sesiones desde el 14/06/2024 con los IDAs
    if ses > 3:
        if date_start > date(2024, 6, 13):
            return pd.DataFrame(columns=['DATETIME', 'MERCADO', 'SESION', 'VALUE'])

    url_id = url + str(id_esios)
    params = {
        'start_date': date_start.strftime('%Y-%m-%dT%H'),
        'end_date': date_end.strftime('%Y-%m-%dT%H')
    }

    data = requests.get(url_id, headers=headers, params=params, verify=False).json()

    df_data = pd.DataFrame(data['indicator']['values'])

    # Mascara de region
    mask_geoname = df_data['geo_name']=='España'
    mask_geoid = df_data['geo_id']==3

    # Filtrado y limpieza
    df_data = df_data[mask_geoname & mask_geoid]
    df_data['MERCADO'] = mkt
    df_data['SESION'] = ses
    df_data = df_data[['datetime', 'MERCADO', 'SESION', 'value']]
    df_data.columns = df_data.columns.str.upper()

    return df_data

***1) Conexion API esios***

In [ ]:
# Token personal, necesrio solicitarlo a REE (yo ya lo tenia)
API_TOKEN = 'a90357a0b0e0f23074624d312cf3a92c3d53afc80fb2ee03bcdb4776216c38ae'

headers = {
    'Host': 'api.esios.ree.es',
    'Accept': 'application/json; application/vnd.esios-api-v1+json',
    'Content-Type': 'application/json',
    'x-api-key': API_TOKEN
}

URL_BASE = 'https://api.esios.ree.es/'
ENDPOINT = 'indicators/'

url = URL_BASE + ENDPOINT

# IDs de los datos requeridos, los podemos obtener directamente del portal de "https://www.esios.ree.es/es/analisis/" y ahorrarnos mas peticiones a la web via API
id_spot = 600
id_idas = [612, 613, 614, 615, 616, 617, 618]
ids_prices = [id_spot] + id_idas

***2) Descarga de los precios***

In [ ]:
# Lista vacia para compilar todos los dataframes y posteriormente concatenar en un solo dataframe
list_dfs_prices = []

# Definir las fechas de inicio y fin
start_date = datetime.strptime(input("Fecha de INICIO (AAAA-MM-DD):"), '%Y-%m-%d').date()
end_date = datetime.strptime(input("Fecha de FIN (AAAA-MM-DD):"), '%Y-%m-%d').date()

In [ ]:
current_date = start_date

while current_date <= end_date: # Bucle para ir haciendo las peticiones de mes en mes y no sobresaturar la demanda de informacion

    next_date = current_date + relativedelta(months=1) - timedelta(seconds=1)

    list_dfs_sesiones = []

    for i in range(len(ids_prices)):

        with warnings.catch_warnings(): # Nos quitamos los 'warnings' para que no aparezcan por pantallas
            warnings.simplefilter("ignore")

            # Condicional para identificar que la sesion "0" es SPOT y las demas son IDAs

            if i==0:
                mkt = 'SPOT'
            else:
                mkt = 'IDA'

            try:
                df_prc_ses = data_indicator_prices(url=url, id_esios=ids_prices[i], mkt= mkt, ses= i, headers= headers, date_start= current_date, date_end= next_date)
                list_dfs_sesiones.append(df_prc_ses.copy(deep=True))
            except Exception as error_prc_ses:
                print(f"Error obteniendo precios para {mkt}_{i} entre {current_date.strftime('%Y-%m-%d')} y {next_date.strftime('%Y-%m-%d')}: ", error_prc_ses)

    # Tras finalziar el bucle de sesiones concateno y agrego a la lista inicial de dataframes
    df_prcs = pd.concat(list_dfs_sesiones, axis=0).reset_index(drop=True)
    list_dfs_prices.append(df_prcs.copy(deep=True))

    # Actualizo la fecha realtiva para obtener el mes siguiente
    current_date += relativedelta(months=1)

df_prcs_final = pd.concat(list_dfs_prices, axis=0).reset_index(drop=True)


***2) Limpieza del dataframe***

In [ ]:
df_prcs_final['DATETIME'] = pd.to_datetime(df_prcs_final['DATETIME'].str.split('+').str[0])
df_prcs_final['FECHA'] = df_prcs_final['DATETIME'].dt.strftime('%Y-%m-%d')
df_prcs_final['PERIODO'] = df_prcs_final['DATETIME'].dt.hour + 1

df_prcs_final.rename(columns={'VALUE': 'PRECIO_EUR'}, inplace=True)
df_prcs_final['PRECIO_EUR'] = df_prcs_final['PRECIO_EUR'].astype(float)

df_prcs_final.set_index('DATETIME', inplace=True)
df_prcs_final = df_prcs_final[['FECHA', 'PERIODO', 'MERCADO', 'SESION', 'PRECIO_EUR']]

df_prcs_final.head()

***3) Almacenamiento de la info en fichero auxiliar***

In [ ]:
df_prcs_final.to_parquet('prcs_spot_ida_spain.parquet', engine='pyarrow')   # Para almacenamiento mas comprimido de grandes cantidades de datos
df_prcs_final.to_csv('prcs_spot_ida_spain.csv', sep=';')